In [1]:
from pathlib import Path
import os
import sys

from dotenv import load_dotenv


def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
        if (p / "backend" / "app").exists():
            return p / "backend"
    raise FileNotFoundError("Could not find repo root with app/ directory")


ROOT_DIR = find_repo_root(Path.cwd().resolve())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

load_dotenv(ROOT_DIR / ".env", override=True)
if os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY"].strip()

print("ROOT_DIR:", ROOT_DIR)
print("KEY LOADED:", bool(os.getenv("OPENAI_API_KEY")))


ROOT_DIR: /Users/user/Desktop/rag_new/backend
KEY LOADED: True


In [2]:
from pathlib import Path
import sys

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
        if (p / "backend" / "app").exists():
            return p / "backend"
    raise FileNotFoundError("Could not find repo root with app/ directory")

ROOT = find_repo_root(Path.cwd().resolve())
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))


In [5]:
from app.rag.pipeline import RAGConfig, run_rag
import json

normalize_keywords = True  # 표준화 키워드 사용 (라우터 매칭 기반)

query = input("질문 입력: ").strip()
if not query:
    raise ValueError("질문을 입력해 주세요.")

config = RAGConfig(top_k=4, normalize_keywords=normalize_keywords)
result = await run_rag(query, config=config)

print("던진 질문:", query)
print("route:", result.get("routing", {}).get("route"))
print(json.dumps({
    "currentSituation": result.get("currentSituation", []),
    "nextStep": result.get("nextStep", []),
    "guidanceScript": result.get("guidanceScript", "")
}, ensure_ascii=False, indent=2))


[rag] route=1.1ms retrieve=411.9ms cards=2281.2ms post=0.2ms total=2694.4ms docs=4 route=card_usage cache=miss
던진 질문: 대출을 좀 해보려고 하는데요
route: card_usage
{
  "currentSituation": [
    {
      "id": "guide_16",
      "title": "장기카드대출(카드론)",
      "keywords": [
        "#대출"
      ],
      "content": "장기카드대출(카드론)은 카드 회원의 신용도와 이용실적에 따라 대출 한도가 정해지며, 2~24개월 동안 원리금균등분할상환, 만기일시상환, 거치후 분할상환 방식으로 상환합니다.",
      "detailContent": "제목: 장기카드대출(카드론)\n\n내용: 카드 회원 본인의 신용도와 카드 이용실적에 따라 카드사에서 대출해 주는 상품입니다. 장기카드대출(카드론) 한도는 회원의 개인신용평점과 과거 카드 결제실적 등에 따라 정해지며, 카드 이용실적이 많고 연체 없이 결제할수록 개인신용평점이 상승합니다. 대출기간은 일반적으로 2~24개월로 단기카드대출(현금서비스)에 비해 장기이며, 상환방식은 원리금균등분할상환 방식(매월 동일한 금액의 원리금 납입), 만기일시상환 방식(매월 이자만 납부하다가 만기일에 원금 납부), 일정기간 거치후 분할상환 방식(거치기간 동안 이자만 납부한 후 나머지 기간 동안 매월 원리금 납부) 등으로 구분됩니다.",
      "relevanceScore": 0.01739344262295082
    },
    {
      "id": "guide_15",
      "title": "단기카드대출(현금서비스)",
      "keywords": [
        "#대출"
      ],
      "content": "단기카드대출(현금서비스)은 카드사의 홈페이지, ARS, CD/ATM을 통해 한도 내에서 편리하게 이용

In [4]:
'''import subprocess, sys

result = subprocess.run(
    [sys.executable, "app/rag/scripts/run_regression.py"],
    capture_output=True,
    text=True,
)
print(result.stdout)
print(result.stderr)'''


'import subprocess, sys\n\nresult = subprocess.run(\n    [sys.executable, "app/rag/scripts/run_regression.py"],\n    capture_output=True,\n    text=True,\n)\nprint(result.stdout)\nprint(result.stderr)'

In [5]:
'''import subprocess, sys
subprocess.run(
    [sys.executable, "app/rag/scripts/run_regression.py"],
    cwd="/Users/user/Desktop/RAG/backend",
)
'''

'import subprocess, sys\nsubprocess.run(\n    [sys.executable, "app/rag/scripts/run_regression.py"],\n    cwd="/Users/user/Desktop/RAG/backend",\n)\n'

In [6]:
from pathlib import Path
import os
import sys

import psycopg2
from dotenv import load_dotenv

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
        if (p / "backend" / "app").exists():
            return p / "backend"
    raise FileNotFoundError("Could not find repo root with app/ directory")

ROOT_DIR = find_repo_root(Path.cwd().resolve())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

load_dotenv(ROOT_DIR / ".env", override=True)

def db_config():
    host = os.getenv("DB_HOST_IP") or os.getenv("DB_HOST")
    cfg = {
        "host": host,
        "dbname": os.getenv("DB_NAME"),
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "port": int(os.getenv("DB_PORT", "0")) or None,
    }
    missing = [k for k, v in cfg.items() if not v]
    if missing:
        raise ValueError(f"Missing DB settings: {missing}")
    return cfg

def fetch(sql, params=None):
    with psycopg2.connect(**db_config()) as conn:
        with conn.cursor() as cur:
            cur.execute(sql, params or ())
            return cur.fetchall()

def show_schema(table: str):
    rows = fetch(
        """
        SELECT column_name, data_type, udt_name
        FROM information_schema.columns
        WHERE table_name = %s
        ORDER BY ordinal_position;
        """,
        (table,),
    )
    return rows

def sample_row(table: str):
    rows = fetch(
        f"""
        SELECT id, metadata, LEFT(content, 200)
        FROM {table}
        ORDER BY id DESC
        LIMIT 1;
        """
    )
    return rows[0] if rows else None

print("ROOT_DIR:", ROOT_DIR)
print("card_tbl schema:", show_schema("card_tbl"))
print("guide_tbl schema:", show_schema("guide_tbl"))

card_sample = sample_row("card_tbl")
guide_sample = sample_row("guide_tbl")
print("card_tbl sample:", card_sample)
print("guide_tbl sample:", guide_sample)

# metadata keys 분포(상위)
def metadata_keys_count(table: str):
    rows = fetch(
        f"""
        SELECT key, COUNT(*) 
        FROM (
            SELECT jsonb_object_keys(metadata) AS key
            FROM {table}
        ) t
        GROUP BY key
        ORDER BY COUNT(*) DESC
        LIMIT 15;
        """
    )
    return rows

print("card_tbl metadata keys:", metadata_keys_count("card_tbl"))
print("guide_tbl metadata keys:", metadata_keys_count("guide_tbl"))


ROOT_DIR: /Users/user/Desktop/rag_new/backend
card_tbl schema: [('id', 'integer', 'int4'), ('content', 'text', 'text'), ('metadata', 'jsonb', 'jsonb'), ('embedding', 'USER-DEFINED', 'vector')]
guide_tbl schema: [('id', 'integer', 'int4'), ('content', 'text', 'text'), ('metadata', 'jsonb', 'jsonb'), ('embedding', 'USER-DEFINED', 'vector')]
card_tbl sample: (216, {'id': 'card_216', 'title': '네이버페이카드포인트 적립 제외 포인트 적립 제외 대상은 아래와 같습니다.', 'category': '연회비', 'card_name': '네이버페이카드'}, '이용거래, 포인트 사용거래 중 포인트금액, 지방세, 국세, 수도요금, 4대보험(국민연금 고용보험건강보험산재보험), 초중고 학교납입금(스쿨뱅킹), 아파트관리비, 도시가스, 전기요금, (신용)로 신한카드 할인서비스(이벤트 포함)를 적용받은 모든 거래 (해당 거래금액 전체), 거래 취소금액')
guide_tbl sample: (491, {'source': 'C:/SKN19/data-preprocessing/data/hyundai\\hyundai_giftcard.json'}, '}\n  },\n  {\n    "id": "환불ㆍ기부 안내_1",\n    "title": "신청 방법 및 시간",\n    "content": "현대카드 홈페이지 및 고객센터를 통해 신청(계좌이체/대체입금/기부 신청 등의 형태로 환불 가능)\\n24시간 신청 가능\\n*평일 오후 6시 이후, 주말 및 공휴일에 환불 신청 시 다음 영업일에 처리",\n    "text"')
card_tbl metadata keys: [('title', 216), (

In [7]:
from pathlib import Path
import os
import sys
import psycopg2
from dotenv import load_dotenv

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
        if (p / "backend" / "app").exists():
            return p / "backend"
    raise FileNotFoundError("Could not find repo root with app/ directory")

ROOT_DIR = find_repo_root(Path.cwd().resolve())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

load_dotenv(ROOT_DIR / ".env", override=True)

def db_config():
    host = os.getenv("DB_HOST_IP") or os.getenv("DB_HOST")
    cfg = {
        "host": host,
        "dbname": os.getenv("DB_NAME"),
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "port": int(os.getenv("DB_PORT", "0")) or None,
    }
    missing = [k for k, v in cfg.items() if not v]
    if missing:
        raise ValueError(f"Missing DB settings: {missing}")
    return cfg

def fetch(sql, params=None):
    with psycopg2.connect(**db_config()) as conn:
        with conn.cursor() as cur:
            cur.execute(sql, params or ())
            return cur.fetchall()

sql = """
SELECT id, metadata->>'id', metadata->>'card_name', metadata->>'category'
FROM card_tbl
WHERE metadata->>'id' = '서울시다둥이행복카드_11';
"""
print("card_tbl:", fetch(sql))

sql = """
SELECT id, metadata->>'id', metadata->>'card_name', metadata->>'category'
FROM guide_tbl
WHERE metadata->>'id' = '서울시다둥이행복카드_11';
"""
print("guide_tbl:", fetch(sql))


card_tbl: []
guide_tbl: []


In [8]:
from pathlib import Path
import os
import sys
import psycopg2
from dotenv import load_dotenv

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
        if (p / "backend" / "app").exists():
            return p / "backend"
    raise FileNotFoundError("Could not find repo root with app/ directory")

ROOT_DIR = find_repo_root(Path.cwd().resolve())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

load_dotenv(ROOT_DIR / ".env", override=True)

def db_config():
    host = os.getenv("DB_HOST_IP") or os.getenv("DB_HOST")
    cfg = {
        "host": host,
        "dbname": os.getenv("DB_NAME"),
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "port": int(os.getenv("DB_PORT", "0")) or None,
    }
    missing = [k for k, v in cfg.items() if not v]
    if missing:
        raise ValueError(f"Missing DB settings: {missing}")
    return cfg

def fetch(sql, params=None):
    with psycopg2.connect(**db_config()) as conn:
        with conn.cursor() as cur:
            cur.execute(sql, params or ())
            return cur.fetchall()

terms = [ "서울시 다산콜센터 "]

def build_like_where(terms):
    clauses = []
    params = []
    for term in terms:
        clauses.append(
            "(content ILIKE %s OR metadata->>'title' ILIKE %s OR "
            "metadata->>'category' ILIKE %s OR metadata->>'category1' ILIKE %s OR "
            "metadata->>'category2' ILIKE %s)"
        )
        params.extend([f"%{term}%"] * 5)
    return " OR ".join(clauses), params

where_sql, params = build_like_where(terms)

sql_card = f"""
SELECT id, metadata->>'id', metadata->>'title', metadata->>'card_name',
       metadata->>'category', metadata->>'category1', metadata->>'category2',
       LEFT(content, 180)
FROM card_tbl
WHERE {where_sql}
ORDER BY id DESC
LIMIT 20;
"""

sql_guide = f"""
SELECT id, metadata->>'id', metadata->>'title', metadata->>'card_name',
       metadata->>'category', metadata->>'category1', metadata->>'category2',
       LEFT(content, 180)
FROM guide_tbl
WHERE {where_sql}
ORDER BY id DESC
LIMIT 20;
"""

print("card_tbl matches:")
print(fetch(sql_card, params))

print("guide_tbl matches:")
print(fetch(sql_guide, params))


card_tbl matches:
[(194, 'card_194', '서울시다둥이행복카드발급 대상- 서울시 거주, 2자녀 이상, 막내 만 18세 이하 가구의 부모 확인 서류 : 주민등록등본 및 가족관계증명서 서울시', '서울시다둥이행복카드', '발급/신청', None, None, '제목: 서울시다둥이행복카드발급 대상- 서울시 거주, 2자녀 이상, 막내 만 18세 이하 가구의 부모 확인 서류 : 주민등록등본 및 가족관계증명서 서울시\n\n내용: 다산콜센터 : 120 - 서울시 거주, 2자녀 이상, 막내 만 18세 이하 가구의 부모 확인 서류 : 주민등록등본 및 가족관계증명서 서울시 다산콜센터 : 120'), (162, 'card_162', '발급 대상', '서울시다둥이행복카드', '발급/신청', None, None, '제목: 발급 대상\n\n내용: - 서울시 거주, 2자녀 이상, 막내 만 18세 이하 가구의 부모  확인 서류 : 주민등록등본 및 가족관계증명서  서울시 다산콜센터 : 120')]
guide_tbl matches:
[]


In [9]:
from pathlib import Path
import os
import sys
import json
import psycopg2
from dotenv import load_dotenv

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
        if (p / "backend" / "app").exists():
            return p / "backend"
    raise FileNotFoundError("Could not find repo root with app/ directory")

ROOT_DIR = find_repo_root(Path.cwd().resolve())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

load_dotenv(ROOT_DIR / ".env", override=True)

def db_config():
    host = os.getenv("DB_HOST_IP") or os.getenv("DB_HOST")
    cfg = {
        "host": host,
        "dbname": os.getenv("DB_NAME"),
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "port": int(os.getenv("DB_PORT", "0")) or None,
    }
    missing = [k for k, v in cfg.items() if not v]
    if missing:
        raise ValueError(f"Missing DB settings: {missing}")
    return cfg

def fetch(sql, params=None):
    with psycopg2.connect(**db_config()) as conn:
        with conn.cursor() as cur:
            cur.execute(sql, params or ())
            return cur.fetchall()

sql = """
SELECT id, metadata
FROM card_tbl
WHERE metadata->>'title' = '발급 대상'
ORDER BY id DESC
LIMIT 5;
"""
rows = fetch(sql)

for row in rows:
    print("id:", row[0])
    print(json.dumps(row[1], ensure_ascii=False, indent=2))


id: 162
{
  "id": "card_162",
  "title": "발급 대상",
  "category": "발급/신청",
  "card_name": "서울시다둥이행복카드"
}


In [10]:
from pathlib import Path
import os
import sys
import json
import psycopg2
from dotenv import load_dotenv

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
        if (p / "backend" / "app").exists():
            return p / "backend"
    raise FileNotFoundError("Could not find repo root with app/ directory")

ROOT_DIR = find_repo_root(Path.cwd().resolve())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

load_dotenv(ROOT_DIR / ".env", override=True)

def db_config():
    host = os.getenv("DB_HOST_IP") or os.getenv("DB_HOST")
    cfg = {
        "host": host,
        "dbname": os.getenv("DB_NAME"),
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "port": int(os.getenv("DB_PORT", "0")) or None,
    }
    missing = [k for k, v in cfg.items() if not v]
    if missing:
        raise ValueError(f"Missing DB settings: {missing}")
    return cfg

def fetch(sql, params=None):
    with psycopg2.connect(**db_config()) as conn:
        with conn.cursor() as cur:
            cur.execute(sql, params or ())
            return cur.fetchall()

# 1) 카드 테이블에서 특정 키워드 매칭 문서 metadata 전체 보기
keyword = "나라사랑"

sql = """
SELECT id, metadata, LEFT(content, 120)
FROM card_tbl
WHERE content ILIKE %s OR metadata->>'title' ILIKE %s
ORDER BY id DESC
LIMIT 10;
"""
rows = fetch(sql, (f"%{keyword}%", f"%{keyword}%"))

for r in rows:
    print("id:", r[0])
    print("metadata:", json.dumps(r[1], ensure_ascii=False, indent=2))
    print("content:", r[2])
    print("-" * 40)

# 2) guide_tbl도 동일 확인
rows = fetch(sql.replace("card_tbl", "guide_tbl"), (f"%{keyword}%", f"%{keyword}%"))

for r in rows:
    print("guide id:", r[0])
    print("metadata:", json.dumps(r[1], ensure_ascii=False, indent=2))
    print("content:", r[2])
    print("-" * 150)


id: 134
metadata: {
  "id": "card_134",
  "title": "발급 안내",
  "category": "발급/신청",
  "card_name": "나라사랑체크카드"
}
content: 제목: 발급 안내

내용: 브랜드: 국내전용  KB국민 나라사랑카드(체크) 종류: KB국민 나라사랑카드(일반형), KB국민 나라사랑카드(ROTC형) * 후불 교통기능: 선택 가능
----------------------------------------
id: 130
metadata: {
  "id": "card_130",
  "title": "군 KT공중전화 할인",
  "category": "혜택/할인",
  "card_name": "나라사랑체크카드"
}
content: 제목: 군 KT공중전화 할인

내용: 「KT나라사랑카드 통화서비스」 및 「KT나라사랑 요금제」 요금 자동 이체 시 10% 환급할인  월 최대 1만원 할인 * 전월 이용실적 관계없이 할인혜택 제공
----------------------------------------
id: 125
metadata: {
  "id": "card_125",
  "title": "포인트리",
  "category": "적립",
  "card_name": "나라사랑체크카드"
}
content: 제목: 포인트리

내용: 나라사랑카드(체크)는 기본 포인트리가 적립되지 않습니다. (단, 스타샵 가맹점에서 제공하는 포인트리는 적립)
----------------------------------------
id: 42
metadata: {
  "id": "card_42",
  "title": "나라사랑포털 관련 문의는 어디에 하나요?",
  "category": "발급/신청",
  "card_name": "나라사랑카드"
}
content: 제목: 나라사랑포털 관련 문의는 어디에 하나요?

내용: 나라사랑포털(www.narasarang.or.kr)은 나라사랑회원 전용사이트로 국방부와 병무청의 통제 하에군인공제회C&

In [11]:
from pathlib import Path
BASE = Path("/Users/user/Desktop/rag_new/backend").resolve()
if not (BASE / "app").exists():
    raise RuntimeError(f"app/ not found under {BASE}")


In [12]:
from pathlib import Path
BASE = Path("/Users/user/Desktop/rag_new/backend").resolve()
print("BASE:", BASE, "app exists:", (BASE / "app").exists())


BASE: /Users/user/Desktop/rag_new/backend app exists: True


In [13]:
from pathlib import Path
import os, re
from collections import Counter

import psycopg2
from dotenv import load_dotenv

ROOT = Path.cwd()
# 노트북이 backend/ 하위에서 열렸든 아니든 대응
if (ROOT / "app").exists():
    BASE = ROOT
elif (ROOT / "backend" / "app").exists():
    BASE = ROOT / "backend"
else:
    raise RuntimeError("app/ 디렉토리를 찾을 수 없습니다.")

load_dotenv(BASE / ".env")

# stopwords (있으면 사용)
stopwords = set()
try:
    import sys
    sys.path.insert(0, str(BASE))
    from app.rag.vocab.rules import STOPWORDS as RULE_STOPWORDS
    stopwords = {str(w).lower() for w in RULE_STOPWORDS}
except Exception:
    pass

def db_config():
    host = os.getenv("DB_HOST_IP") or os.getenv("DB_HOST")
    cfg = {
        "host": host,
        "dbname": os.getenv("DB_NAME"),
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "port": int(os.getenv("DB_PORT", "0")) or None,
    }
    missing = [k for k, v in cfg.items() if not v]
    if missing:
        raise ValueError(f"Missing DB settings: {missing}")
    return cfg

TOKEN_RE = re.compile(r"[A-Za-z0-9가-힣]+")

PHRASES = [
    "주소", "주소 변경", "이사", "통보",
    "납부", "대금", "결제일", "자동이체", "연체", "미납",
    "해지", "정지", "한도", "상향", "증액",
    "재발급", "분실", "도난", "교통",
    "연회비", "청구", "할인", "적립", "혜택",
]

def tokenize(text: str):
    if not text:
        return []
    tokens = TOKEN_RE.findall(text)
    out = []
    for tok in tokens:
        t = tok.lower()
        if len(t) < 2 or t.isdigit():
            continue
        if t in stopwords:
            continue
        out.append(t)
    return out

def scan_table(table: str, include_content=False):
    counter = Counter()
    phrase_counter = Counter()
    with psycopg2.connect(**db_config()) as conn:
        cur = conn.cursor(name=f"scan_{table}")
        cur.itersize = 1000
        if include_content:
            cur.execute(f"SELECT metadata->>'title', content FROM {table}")
            for title, content in cur:
                text = " ".join(filter(None, [title, (content or "")[:300]]))
                for tok in tokenize(text):
                    counter[tok] += 1
                text_lower = text.lower()
                for phrase in PHRASES:
                    if phrase in text_lower:
                        phrase_counter[phrase] += 1
        else:
            cur.execute(f"SELECT metadata->>'title' FROM {table}")
            for (title,) in cur:
                text = title or ""
                for tok in tokenize(text):
                    counter[tok] += 1
                text_lower = text.lower()
                for phrase in PHRASES:
                    if phrase in text_lower:
                        phrase_counter[phrase] += 1
        cur.close()
    return counter, phrase_counter

for table in ("card_tbl", "guide_tbl"):
    counter, phrase_counter = scan_table(table, include_content=False)  # 필요하면 True로
    print(f"\n[{table}] top tokens (title 기준)")
    for token, count in counter.most_common(40):
        print(f"- {token}: {count}")
    print(f"\n[{table}] phrase hits (title 기준)")
    for phrase, count in phrase_counter.most_common():
        print(f"- {phrase}: {count}")


RuntimeError: app/ 디렉토리를 찾을 수 없습니다.